In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import datetime, date

In [5]:
# Настройка вывода данных
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

## Подготовка данных.
### Сбор сырых(исходных) данных.

In [43]:
df_f = pd.read_csv("./data/weather_01-01-2017_31-12-2020.csv")
df_f['Местное время в Абакане (аэропорт)'] = pd.to_datetime(df_f['Местное время в Абакане (аэропорт)'], format='%d.%m.%Y %H:%M')
df_f

,Местное время в Абакане (аэропорт),Темп.,мм.рт.ст,Влажность,Направление ветра,Скорость ветра
0,2020-12-31 23:30:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
1,2020-12-31 23:00:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
2,2020-12-31 22:30:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
3,2020-12-31 22:00:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
4,2020-12-31 21:30:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
...,...,...,...,...,...,...
65524,2017-02-13 22:30:00,"-21,0","750,6",77.0,"Ветер, дующий с северо-запада",1
65525,2017-02-13 22:00:00,"-21,0","749,9",77.0,"Ветер, дующий с северо-запада",1
65526,2017-02-13 21:30:00,"-20,0","750,0",77.0,"Ветер, дующий с севера",1
65527,2017-02-13 21:00:00,"-21,0","749,9",77.0,"Ветер, дующий с севера",1


In [61]:
df_f.loc[df_f['Местное время в Абакане (аэропорт)'].dt.year == 2020]

,Местное время в Абакане (аэропорт),Темп.,мм.рт.ст,Влажность,Направление ветра,Скорость ветра
0,2020-12-31 23:30:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
1,2020-12-31 23:00:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
2,2020-12-31 22:30:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
3,2020-12-31 22:00:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
4,2020-12-31 21:30:00,"-22,0","750,5",77.0,"Штиль, безветрие",0
...,...,...,...,...,...,...
17336,2020-01-01 02:00:00,"-11,0","743,7",85.0,"Штиль, безветрие",0
17337,2020-01-01 01:30:00,"-11,0","742,9",85.0,"Штиль, безветрие",0
17338,2020-01-01 01:00:00,"-11,0","742,9",85.0,"Штиль, безветрие",0
17339,2020-01-01 00:30:00,"-11,0","742,9",79.0,"Штиль, безветрие",0


### Подготовка данных

In [ ]:
# Удаление лишних столбцов и колонок
df_clear = df_f.drop(index=[0, 1, 269, 270, 271], columns='Unnamed: 3389').reset_index().drop(columns='index')